In [2813]:
import abc
from collections import defaultdict
from dataclasses import dataclass, field
from enum import IntEnum
from numbers import Number
from typing import Optional, Iterator, Sequence, TypeVar, Set, Mapping, Union, FrozenSet, Iterable

import clingo
import clingraph
import more_itertools
from clingraph import Factbase, compute_graphs

In [2814]:
def powerset(iterable):
    for i in more_itertools.powerset(iterable):
        yield set(i)

In [2815]:
def frozen_powerset(iterable):
    for i in more_itertools.powerset(iterable):
        yield frozenset(i)

In [2816]:
def asp_solve(programs,
              ctl: Optional[clingo.Control] = None,
              parts=(('base', ()),),
              context=None,
              report=False,
              report_models=True,
              report_result=True,
              symbol_sep=' ',
              model_sep='\n'
              ) -> Iterator[Sequence[clingo.Symbol]]:
    if ctl is None:
        ctl = clingo.Control(logger=lambda *args, **kwargs: None)
        ctl.configuration.solve.models = 0
    if programs:
        if isinstance(programs, str):
            ctl.add('base', [], programs)
        else:
            for program in programs:
                if isinstance(program, ASPProgram):
                    ctl.add('base', [], str(program))
                elif isinstance(program, str):
                    ctl.add('base', [], program)

    ctl.ground(parts, context=context)
    with ctl.solve(yield_=True) as solve_handle:
        models = 0
        for model in solve_handle:
            symbols = sorted(model.symbols(shown=True))
            if report and report_models:
                print("Answer {}:".format(model.number), end=' ')
                print("{",
                      symbol_sep.join(map(str, sorted(symbols))), "}", sep=symbol_sep, end=model_sep)
            models += 1
            yield symbols
        if report and report_result:
            solve_result = solve_handle.get()
            print(solve_result, end='')
            if solve_result.satisfiable:
                print(" {}{}".format(models, '' if solve_result.exhausted else '+'))
            else:
                print()


In [2817]:

def draw_graph(programs,
               ctl: Optional[clingo.Control] = None,
               parts=(('base', ()),)):
    fb = Factbase()
    if ctl is None:
        ctl = clingo.Control(logger=lambda *args, **kwargs: None)
        ctl.configuration.solve.models = 0
    ctl.add('base', [], '\n'.join(programs))
    ctl.ground(parts, clingraph.clingo_utils.ClingraphContext())
    with ctl.solve(yield_=True) as solve_handle:
        for model in solve_handle:
            fb.add_model(model)
            break
    return compute_graphs(fb)


In [2818]:

ForwardASPSymbol = TypeVar('ForwardASPSymbol', bound='ASPSymbol')
ForwardASPVariable = TypeVar('ForwardASPVariable', bound='ASPVariable')


class ASPSymbol(abc.ABC):

    @staticmethod
    def from_clingo_symbol(symbol: clingo.Symbol) -> ForwardASPSymbol:
        if symbol.type is clingo.SymbolType.Number:
            return ASPTerm(ASPIntegerConstant(symbol.number))
        elif symbol.type is clingo.SymbolType.String:
            return ASPTerm(ASPStringConstant(symbol.string))
        elif symbol.type is clingo.SymbolType.Function:
            arguments = tuple(ASPFunction.from_clingo_symbol(argument) for argument in symbol.arguments)
            name = symbol.name
            if symbol.negative:
                name = "-{}".format(name)
            return ASPFunction(name, arguments)
        else:
            assert False, "Unhandled clingo.Symbol {} with type {}.".format(symbol, symbol.type.name)


@dataclass(order=True, frozen=True)
class ASPStringConstant:
    string: str = field(default="")

    def __str__(self):
        return '"{}"'.format(self.string)


@dataclass(order=True, frozen=True)
class ASPIntegerConstant:
    number: int = field(default=0)

    def __str__(self):
        return str(self.number)


@dataclass(order=True, frozen=True)
class ASPTerm(ASPSymbol):
    constant: Union[ASPStringConstant, ASPIntegerConstant] = field(default_factory=ASPIntegerConstant)

    def __str__(self):
        return str(self.constant)

    @staticmethod
    def zero():
        return ASPTerm(ASPIntegerConstant(0))

    @staticmethod
    def one():
        return ASPTerm(ASPIntegerConstant(1))


ForwardASPTopLevelSymbol = TypeVar('ForwardASPTopLevelSymbol', bound='ASPTopLevelSymbol')


class ASPTopLevelSymbol(ASPSymbol, abc.ABC):

    def __neg__(self):
        raise NotImplementedError

    @property
    def signature(self) -> str:
        return "{}/{}.".format(self.function_name, self.arity)

    @property
    @abc.abstractmethod
    def function_name(self) -> str:
        raise NotImplementedError

    @property
    def arity(self) -> int:
        return len(self.function_arguments)

    @property
    @abc.abstractmethod
    def function_arguments(self) -> Sequence[ASPSymbol]:
        raise NotImplementedError

    def match(self, other: ForwardASPTopLevelSymbol) -> bool:
        return self.function_name == other.function_name and len(self.function_arguments) == len(
            other.function_arguments)



In [2819]:
def asp_evaluate(*programs, report=False) -> Iterator[FrozenSet[ASPTopLevelSymbol]]:
    for answer_set in asp_solve(programs=programs, report=report):
        yield frozenset(ASPFunction.from_clingo_symbol(symbol) for symbol in answer_set)


In [2820]:
@dataclass(order=True, frozen=True)
class ASPFunction(ASPTopLevelSymbol):
    name: Optional[str] = field(default=None)
    arguments: Sequence[ASPSymbol] = field(default_factory=tuple)

    @property
    def function_name(self) -> str:
        return self.name

    @property
    def function_arguments(self) -> Sequence[ASPSymbol]:
        return self.arguments

    @property
    def arity(self) -> int:
        return len(self.arguments)

    def __str__(self):
        if self.name is None:
            return '({})'.format(','.join(map(str, self.arguments)))
        elif not self.arguments:
            return self.name
        else:
            return '{}({})'.format(self.name, ','.join(map(str, self.arguments)))

    def __neg__(self):
        if self.name.startswith('-'):
            return ASPFunction(self.name[1:], self.arguments)
        else:
            return ASPFunction('-{}'.format(self.name), self.arguments)


ForwardASPAtom = TypeVar('ForwardASPAtom', bound='ASPAtom')


@dataclass(order=True, frozen=True)
class ASPAtom:
    symbol: ASPTopLevelSymbol = field(default_factory=ASPFunction)

    @property
    def signature(self) -> str:
        return self.symbol.signature

    def match(self, other: ForwardASPAtom) -> bool:
        return self.symbol.function_name == other.symbol.function_name and len(self.symbol.function_arguments) == len(
            other.symbol.function_arguments)

    def __str__(self):
        return str(self.symbol)

    def __neg__(self):
        return ASPAtom(-self.symbol)


@dataclass(order=True, frozen=True)
class ASPClauseElement(abc.ABC):

    @property
    @abc.abstractmethod
    def signature(self) -> str:
        raise NotImplementedError

    @property
    @abc.abstractmethod
    def is_pos(self):
        raise NotImplementedError

    @property
    @abc.abstractmethod
    def is_neg(self):
        raise NotImplementedError

    def __neg__(self):
        raise NotImplementedError

    def __abs__(self):
        raise NotImplementedError


class ASPHeadClauseElement(ASPClauseElement):
    pass


class ASPLiteral(ASPHeadClauseElement):

    @property
    @abc.abstractmethod
    def signature(self) -> str:
        raise NotImplementedError

    @property
    @abc.abstractmethod
    def is_pos(self) -> bool:
        raise NotImplementedError

    @property
    @abc.abstractmethod
    def is_neg(self) -> bool:
        raise NotImplementedError


class ASPSign(IntEnum):
    NoSign = 0
    Negation = 1

    def __str__(self):
        if self is ASPSign.NoSign:
            return ''
        elif self is ASPSign.Negation:
            return 'not'
        else:
            assert False, 'Unknown IntEnum {} = {}.'.format(self.name, self.value)


@dataclass(order=True, frozen=True)
class ASPBasicLiteral(ASPLiteral):
    sign: ASPSign = ASPSign.NoSign
    atom: ASPAtom = field(default_factory=ASPAtom)

    @property
    def is_pos(self) -> bool:
        return self.sign is ASPSign.NoSign

    @property
    def is_neg(self) -> bool:
        return self.sign is ASPSign.Negation

    @property
    def signature(self) -> str:
        return self.atom.signature

    def __str__(self):
        if self.sign is ASPSign.NoSign:
            return "{}".format(self.atom)
        else:
            return "{} {}".format(self.sign, self.atom)

    def __neg__(self):
        return ASPBasicLiteral(ASPSign((self.sign ^ 1) % 2), self.atom)

    def __invert__(self):
        return ASPBasicLiteral(sign=self.sign, atom=-self.atom)

    def __abs__(self):
        return ASPBasicLiteral(ASPSign.NoSign, self.atom)

    def as_classical_atom(self):
        return ClassicalAtom(str(self.atom.symbol))

    @staticmethod
    def make_literal(name: Optional[str], *arguments: Union[str, int, ASPFunction]):
        function_arguments = []
        for arg_ in arguments:
            if isinstance(arg_, str):
                arg = ASPFunction(arg_)
            elif isinstance(arg_, int):
                arg = ASPTerm(ASPIntegerConstant(arg_))
            else:
                arg = arg_
            function_arguments.append(arg)
        return ASPBasicLiteral(atom=ASPAtom(ASPFunction(name=name, arguments=tuple(function_arguments))))


@dataclass(order=True, frozen=True)
class ASPConditionalLiteral(ASPLiteral):
    literal: ASPBasicLiteral = field(default_factory=ASPBasicLiteral)
    conditions: Sequence[ASPClauseElement] = field(default_factory=tuple)

    @property
    def signature(self) -> str:
        return self.literal.signature

    @property
    def is_pos(self) -> bool:
        return self.literal.is_pos

    @property
    def is_neg(self) -> bool:
        return self.literal.is_neg

    def __neg__(self):
        return ASPConditionalLiteral(-self.literal, self.conditions)

    def __abs__(self):
        return ASPConditionalLiteral(abs(self.literal), self.conditions)

    def __str__(self):
        return "{} : {}".format(self.literal, ','.join(map(str, self.conditions)))


@dataclass(order=True, frozen=True)
class ASPDirective(ASPHeadClauseElement):
    name: str
    arguments: Sequence[Union[Sequence[ASPSymbol], ASPSymbol]] = field(default_factory=tuple)

    @property
    def is_true(self) -> bool:
        return self.name == 'true'

    @property
    def is_pos(self) -> bool:
        return self.is_true

    @property
    def is_false(self) -> bool:
        return self.name == 'false'

    @property
    def is_neg(self) -> bool:
        return self.is_false

    @property
    def is_forall(self) -> bool:
        return self.name == 'forall'

    @property
    def is_show(self) -> bool:
        return self.name == 'show'

    @property
    def signature(self) -> str:
        return '#{}/{}.'.format(self.name, len(self.arguments))

    def __abs__(self):
        raise NotImplementedError

    def __neg__(self):
        if self.name == 'true':
            return ASPDirective.false()
        elif self.name == 'false':
            return ASPDirective.true()
        else:
            raise NotImplementedError

    def __invert__(self):
        return -self

    def __str__(self):
        if not self.arguments:
            return "#{}".format(self.name)
        else:
            return "#{}({})".format(self.name, ','.join(map(str, self.arguments)))

    @staticmethod
    def true():
        return ASPDirective('true')

    @staticmethod
    def false():
        return ASPDirective('false')

    @staticmethod
    def show(*args):
        return ASPDirective('show', arguments=args)


@dataclass(order=True, frozen=True)
class ASPRule(abc.ABC):
    head: Optional[ASPHeadClauseElement] = field(default=None)
    body: Optional[Sequence[ASPClauseElement]] = field(default=None)

    @property
    @abc.abstractmethod
    def head_signature(self) -> Union[str, Set[str]]:
        raise NotImplementedError

    @staticmethod
    def fmt_body(body: Sequence[ASPClauseElement]):
        return ', '.join(map(str, body))

    def reduct(self, elems: Set[ASPTopLevelSymbol]):
        if self.body is not None:
            if any(literal.is_neg and literal.atom.symbol in elems for literal in self.body):
                return None
        raise NotImplementedError


@dataclass(order=True, frozen=True)
class ASPNormalRule(ASPRule):
    head: ASPBasicLiteral = field(default_factory=ASPBasicLiteral)
    body: Sequence[ASPClauseElement] = field(default_factory=tuple)

    @property
    def head_signature(self) -> str:
        return self.head.signature

    def __str__(self):
        if self.body:
            return "{} :- {}.".format(self.head, ASPRule.fmt_body(self.body))
        else:
            return "{}.".format(self.head)

    def reduct(self, elems: Set[ASPTopLevelSymbol]):
        if any(literal.is_neg and literal.atom.symbol in elems for literal in self.body):
            return None
        return ASPNormalRule(self.head, tuple(literal for literal in self.body if literal.is_pos))


@dataclass(order=True, frozen=True)
class ASPIntegrityConstraint(ASPRule):
    body: Sequence[ASPClauseElement] = field(default_factory=tuple)
    head: ASPDirective = field(default_factory=ASPDirective.false, init=False)

    @property
    def head_signature(self) -> str:
        return '#false/0.'

    def __str__(self):
        if self.body:
            return ":- {}.".format(ASPRule.fmt_body(self.body))
        else:
            return ":-."

    def reduct(self, elems: Set[ASPTopLevelSymbol]):
        if any(literal.is_neg and literal.atom.symbol in elems for literal in self.body):
            return None
        return ASPIntegrityConstraint(tuple(literal for literal in self.body if literal.is_pos))


@dataclass(order=True, frozen=True)
class ASPDisjunctiveRule(ASPRule):
    head: Sequence[ASPLiteral] = field(default_factory=tuple)
    body: Sequence[ASPClauseElement] = field(default_factory=tuple)

    @property
    def head_signature(self) -> Set[str]:
        return {h.signature for h in self.head}

    def __str__(self):
        return "{} :- {}.".format('; '.join(map(str, self.head)), ASPRule.fmt_body(self.body))

    def reduct(self, elems: Set[ASPTopLevelSymbol]):
        if any(literal.is_neg and literal.atom.symbol in elems for literal in self.body):
            return None
        return ASPDisjunctiveRule(
            head=tuple(literal for literal in self.head if literal.is_pos or (literal.atom.symbol not in elems)),
            body=tuple(literal for literal in self.body if literal.is_pos))


@dataclass(order=True, frozen=True)
class ASPChoiceRule(ASPRule):
    lower: Optional[ASPSymbol] = field(default=None)
    head: Sequence[ASPLiteral] = field(default_factory=tuple)
    upper: Optional[ASPSymbol] = field(default=None)
    body: Sequence[ASPClauseElement] = field(default_factory=tuple)

    @property
    def head_signature(self) -> Union[str, Set[str]]:
        return {h.signature for h in self.head}

    def __str__(self):
        lower = ""
        if self.lower is not None:
            lower = "{} <=".format(self.lower)
        upper = ""
        if self.upper is not None:
            upper = "<= {}".format(self.upper)
        body = ""
        if self.body:
            body = " :- {}".format(ASPRule.fmt_body(self.body))
        return "{}{}{}{}{}{}".format(lower, '{', "; ".join(map(str, self.head)), '}', upper, body)


@dataclass(order=True, frozen=True)
class ASPProgram:
    rules: Sequence[ASPRule] = field(default_factory=tuple)

    def fmt(self, sep='\n'):
        return sep.join(map(str, self.rules))

    def __str__(self):
        return self.fmt(' ')

    def reduct(self, elems: Set[ASPTopLevelSymbol]):
        return ASPProgram(tuple(filter(lambda r: r is not None, (rule.reduct(elems) for rule in self.rules))))

In [2821]:


@dataclass(frozen=True, order=True)
class ClassicalAtom:
    symbol: str

    def __neg__(self):
        if self.is_complement:
            return ClassicalAtom(self.symbol[1:])
        else:
            return ClassicalAtom('-{}'.format(self.symbol))

    def __abs__(self):
        if self.is_complement:
            return -self
        return self

    def __str__(self):
        return self.symbol

    @property
    def is_complement(self) -> bool:
        return self.symbol.startswith('-')

    def as_asp_top_level_symbol(self) -> ASPTopLevelSymbol:
        return ASPFunction(self.symbol)

    def as_asp_atom(self) -> ASPAtom:
        return ASPAtom(self.as_asp_top_level_symbol())

    def as_asp_literal(self) -> ASPBasicLiteral:
        return ASPBasicLiteral(atom=self.as_asp_atom())


ClassicalAlphabet = Set[ClassicalAtom]
ClassicalValuation = Mapping[ClassicalAtom, bool]


@dataclass(frozen=True, order=True)
class ClassicalLiteral:
    atom: ClassicalAtom
    sign: bool = field(default=True)

    def __str__(self):
        sign_str = ""
        if not self.sign:
            sign_str = "¬"
        return "{}{}".format(sign_str, self.atom)

    def __repr__(self):
        return str(self)

    def __neg__(self):
        return ClassicalLiteral(self.atom, not self.sign)

    def __and__(self, other):
        left = ClassicalFormula(self)
        right = other
        if isinstance(other, ClassicalLiteral):
            right = ClassicalFormula(right)
        return ClassicalFormula(left, ClassicalConnective.And, right)

    def __or__(self, other):
        left = ClassicalFormula(self)
        right = other
        if isinstance(other, ClassicalLiteral):
            right = ClassicalFormula(right)
        return ClassicalFormula(left, ClassicalConnective.Or, right)

    def __rshift__(self, other):
        left = ClassicalFormula(self)
        right = other
        if isinstance(other, ClassicalLiteral):
            right = ClassicalFormula(right)
        return ClassicalFormula(left, ClassicalConnective.Implies, right)

    def __truediv__(self, other):
        left = ClassicalFormula(self)
        right = other
        if isinstance(other, ClassicalLiteral):
            right = ClassicalFormula(other)
        return Normally(left, right)


@dataclass(frozen=True, order=True)
class ClassicalTop(ClassicalLiteral):
    atom: ClassicalAtom = field(default=ClassicalAtom('⊤'), init=False)
    sign: bool = field(default=True, init=False)

    def __str__(self):
        return str(self.atom)

    def __repr__(self):
        return str(self)

    def __neg__(self):
        return ClassicalBot()

    def __and__(self, other):
        if isinstance(other, ClassicalLiteral):
            return ClassicalFormula(other)
        return other

    def __or__(self, other):
        return ClassicalFormula(ClassicalTop())


@dataclass(frozen=True, order=True)
class ClassicalBot(ClassicalLiteral):
    atom: ClassicalAtom = field(default=ClassicalAtom('⊥'), init=False)
    sign: bool = field(default=False, init=False)

    def __neg__(self):
        return ClassicalTop()

    def __str__(self):
        return str(self.atom)

    def __and__(self, other):
        return ClassicalFormula(ClassicalBot())

    def __or__(self, other):
        if isinstance(other, ClassicalLiteral):
            return ClassicalFormula(other)
        return other


class ClassicalConnective(IntEnum):
    And = 0
    Or = 1
    Implies = 2

    def __str__(self):
        if self is ClassicalConnective.And:
            return "∧"
        elif self is ClassicalConnective.Or:
            return "∨"
        elif self is ClassicalConnective.Implies:
            return "→"
        else:
            assert False, "Unhandled Connective.__str__: {} = {}".format(self.name, self.value)

    def evaluate(self, left: bool, right: bool):
        if self is ClassicalConnective.And:
            return left and right
        elif self is ClassicalConnective.Or:
            return left or right
        elif self is ClassicalConnective.Implies:
            return not left or right
        else:
            assert False, "Unhandled Connective.evaluate: {} = {}".format(self.name, self.value)


ForwardClassicalFormula = TypeVar('ForwardClassicalFormula', bound='ClassicalFormula')


@dataclass(frozen=True, order=True)
class ClassicalFormula:
    left: Union[ForwardClassicalFormula, ClassicalLiteral]
    connective: Optional[ClassicalConnective] = field(default=None)
    right: Union[ForwardClassicalFormula, None] = field(default=None)

    def __str__(self):
        left_str = str(self.left)
        connective_str = ""
        if self.connective is not None:
            connective_str = " {}".format(self.connective)
            if isinstance(self.left,
                          ClassicalFormula) and self.left.connective is not None and self.left.connective > self.connective:
                left_str = "({})".format(left_str)
        right_str = ""
        if self.right is not None:
            if self.right.left == ClassicalBot() and self.right.right is None and self.connective is ClassicalConnective.Implies:
                left_str = "¬({})".format(left_str)
                connective_str = ""
            else:
                right_str = " {}".format(self.right)
            if isinstance(self.right,
                          ClassicalFormula) and self.right.connective is not None and self.right.connective > self.connective:
                left_str = "({})".format(left_str)
        return "{}{}{}".format(left_str, connective_str, right_str)

    def __repr__(self):
        return str(self)

    def __neg__(self):
        if self.connective is not None and self.right is None:
            raise TypeError("Formula.connective present, despite Formula.right missing.")
        elif self.connective is None and self.right is not None:
            raise TypeError("Formula.connective missing, despite Formula.right present.")

        if self.connective is None and self.right is None:
            return ClassicalFormula(-self.left)
        elif self.connective is ClassicalConnective.And:
            return ClassicalFormula(-self.left, ClassicalConnective.Or, -self.right)
        elif self.connective is ClassicalConnective.Or:
            return ClassicalFormula(-self.left, ClassicalConnective.And, -self.right)
        elif self.connective is ClassicalConnective.Implies:
            if self.right.left == ClassicalBot() and self.right.right is None:
                return self.left
            return ClassicalFormula(self, ClassicalConnective.Implies, ClassicalFormula(ClassicalBot()))
        else:
            assert False, "Unknown Formula.connective. {} = {}.".format(self.connective.name, self.connective.value)

    def __and__(self, other):
        left = self
        right = other
        if isinstance(other, ClassicalLiteral):
            right = ClassicalFormula(right)
        if left.is_top:
            return right
        elif right.is_top:
            return left
        if left.is_bot:
            return left
        elif right.is_bot:
            return right
        return ClassicalFormula(left, ClassicalConnective.And, right)

    def __or__(self, other):
        left = self
        right = other
        if isinstance(other, ClassicalLiteral):
            right = ClassicalFormula(right)
        if left.is_top:
            return left
        elif right.is_top:
            return right
        if left.is_bot:
            return right
        elif right.is_bot:
            return left
        return ClassicalFormula(left, ClassicalConnective.Or, right)

    def __rshift__(self, other):
        left = self
        right = other
        if isinstance(other, ClassicalLiteral):
            right = ClassicalFormula(right)
        return ClassicalFormula(left, ClassicalConnective.Implies, right)

    def __truediv__(self, other):
        left = self
        right = other
        if isinstance(other, ClassicalLiteral):
            right = ClassicalFormula(other)
        return Normally(left, right)

    def __call__(self, valuation: Optional[ClassicalValuation] = None) -> bool:
        return self.evaluate(valuation)

    @property
    def literals(self) -> Set[ClassicalLiteral]:
        literals = set()
        if isinstance(self.left, ClassicalLiteral):
            if not isinstance(self.left, ClassicalTop) and not isinstance(self.left, ClassicalBot):
                literals.add(self.left)
        else:
            assert isinstance(self.left, ClassicalFormula), "Unknown type for Formula.right. {}: {}".format(
                type(self.left).__name__, self.left)
            literals.update(self.left.literals)
        if self.right is not None:
            assert isinstance(self.right, ClassicalFormula), "Unknown type for Formula.right. {}: {}".format(
                type(self.right).__name__, self.right)
            literals.update(self.right.literals)
        return literals

    @property
    def atoms(self) -> Set[ClassicalAtom]:
        return {literal.atom for literal in self.literals}

    @property
    def is_top(self) -> bool:
        return self.right is None and isinstance(self.left, ClassicalTop)

    @property
    def is_bot(self) -> bool:
        return self.right is None and isinstance(self.left, ClassicalBot)

    def evaluate(self, valuation: Optional[ClassicalValuation] = None) -> bool:
        if isinstance(self.left, ClassicalLiteral):
            value_left = self.__evaluate_literal(self.left, valuation)
        else:
            assert isinstance(self.left, ClassicalFormula), "Unknown type for Formula.left. {}: {}".format(
                type(self.left).__name__, self.left)
            value_left = self.left.evaluate(valuation)
        if self.connective is not None and self.right is None:
            raise TypeError("Formula.connective present, despite Formula.right missing.")
        elif self.connective is None and self.right is not None:
            raise TypeError("Formula.connective missing, despite Formula.right present.")

        if self.connective is None and self.right is None:
            return value_left
        else:
            assert isinstance(self.right, ClassicalFormula), "Unknown type for Formula.right. {}: {}".format(
                type(self.right).__name__, self.right)
            value_right = self.right.evaluate(valuation)

            return self.connective.evaluate(value_left, value_right)

    def __evaluate_literal(self, literal: ClassicalLiteral, valuation: Optional[ClassicalValuation] = None) -> bool:
        if isinstance(literal, ClassicalTop) or isinstance(literal, ClassicalBot):
            return literal.sign
        else:
            # get assigned truth value of atom (per default false) and flip the result if negated
            return valuation is not None and bool(valuation.get(literal.atom, False) ^ (not literal.sign))

    def set_to_bot(self, *atoms: ClassicalAtom) -> ForwardClassicalFormula:
        if isinstance(self.left, ClassicalLiteral):
            assert self.connective is None
            assert self.right is None
            if self.left.atom in atoms:
                if self.left.sign:
                    return ClassicalFormula(ClassicalBot())
                else:
                    return ClassicalFormula(ClassicalTop())
            else:
                return self
        else:
            left = self.left.set_to_bot(*atoms)
            right = self.right
            if right is not None:
                right = self.right.set_to_bot(*atoms)
            return ClassicalFormula(left, self.connective, right)

    def as_asp_bodies(self) -> Sequence[Sequence[ASPBasicLiteral]]:
        bodies: Sequence[Sequence[ASPBasicLiteral]] = [()]
        if isinstance(self.left, ClassicalLiteral):
            if isinstance(self.left, ClassicalTop):
                bodies = ((ASPDirective.true(),),)
            elif isinstance(self.left, ClassicalBot):
                bodies = ((ASPDirective.false(),),)
            else:
                literal = ASPBasicLiteral(sign=ASPSign(int(not self.left.sign)),
                                          atom=ASPAtom(ASPFunction(self.left.atom.symbol)))
                bodies = ((literal,),)
        elif self.right is None:
            bodies = self.left.as_asp_bodies()
        elif self.right is not None:
            bodies_left = self.left.as_asp_bodies()
            bodies_right = self.right.as_asp_bodies()
            bodies_ = []
            if self.connective is ClassicalConnective.Or:
                for body in bodies:
                    for body_left in bodies_left:
                        bodies_.append((*body, *body_left))
                    for body_right in bodies_right:
                        bodies_.append((*body, *body_right))
            elif self.connective is ClassicalConnective.And:
                for body in bodies:
                    for body_left in bodies_left:
                        for body_right in bodies_right:
                            bodies_.append((*body, *body_left, *body_right))
            else:
                assert self.connective is ClassicalConnective.Implies
                if self.right.is_bot:
                    if self.left.connective is not None and self.left.connective is ClassicalConnective.Implies:
                        bodies_ = (-((-self.left.left) | self.left.right)).as_asp_bodies()
                    else:
                        bodies_ = (-self.left).as_asp_bodies()
                else:
                    for body in bodies:
                        for body_left in bodies_left:
                            bodies_.append((*body, *(-literal for literal in body_left)))
                        for body_right in bodies_right:
                            bodies_.append((*body, *body_right))
            bodies = bodies_
        else:
            assert False
        return bodies



In [2822]:


def all_valuations(alphabet: ClassicalAlphabet, complete: bool = False) -> Iterator[ClassicalValuation]:
    subsets = powerset(alphabet)
    for subset in subsets:
        valuation = defaultdict(lambda: False)
        for atom in subset:
            valuation[atom] = True
        if complete:
            for atom in alphabet:
                if atom not in subset:
                    valuation[atom] = False
        yield valuation


def models(formulas: Set[ClassicalFormula], alphabet: Optional[ClassicalAlphabet] = None) -> Iterator[
    ClassicalValuation]:
    if alphabet is None:
        alphabet = {atom for formula in formulas for atom in formula.atoms}
    for valuation in all_valuations(alphabet):
        if all(formula.evaluate(valuation) for formula in formulas):
            yield valuation


def sat(formulas: Set[ClassicalFormula], alphabet: Optional[ClassicalAlphabet] = None) -> bool:
    model = next(models(formulas, alphabet), None)
    return model is not None


def unsat(formulas: Set[ClassicalFormula], alphabet: Optional[ClassicalAlphabet] = None) -> bool:
    return not sat(formulas, alphabet)


def entails(formulas: Set[ClassicalFormula], formula: ClassicalFormula) -> bool:
    return unsat(formulas | {-formula})


def valid(formulas: Set[ClassicalFormula], alphabet: Optional[ClassicalAlphabet] = None) -> bool:
    if alphabet is None:
        alphabet = {atom for formula in formulas for atom in formula.atoms}
    for valuation in all_valuations(alphabet):
        if any(not formula.evaluate(valuation) for formula in formulas):
            return False
    return True


In [2823]:
@dataclass(frozen=True, order=True)
class Normally:
    left: ClassicalFormula
    right: ClassicalFormula

    def __str__(self):
        if self.is_classical:
            return str(-self.left)
        return "{} |~ {}".format(self.left, self.right)

    def __repr__(self):
        return str(self)

    @property
    def literals(self) -> Set[ClassicalLiteral]:
        return {literal for formula in (self.left, self.right) for literal in formula.literals}

    @property
    def atoms(self) -> Set[ClassicalAtom]:
        return {atom for formula in (self.left, self.right) for atom in formula.atoms}

    @property
    def is_classical(self) -> bool:
        return self.right.left == ClassicalBot() and self.right.right is None

    def materialize(self):
        if self.is_classical:
            if self.left.right is not None and self.left.right.left == ClassicalBot() and self.left.right.right is None:
                return self.left.left
            return -self.left
        return self.left >> self.right


KnowledgeBase = Set[Normally]

In [2824]:
def materialized(k: Set[Normally]) -> Set[ClassicalFormula]:
    return {statement.materialize() for statement in k}

In [2825]:
def evaluate(k: Set[ClassicalFormula], v: ClassicalValuation) -> bool:
    return all(s(v) for s in k)

In [2826]:
a__ = ASPBasicLiteral.make_literal('a')
a_ = a__.as_classical_atom()
a = ClassicalFormula(ClassicalLiteral(a_))
b__ = ASPBasicLiteral.make_literal('b')
b_ = b__.as_classical_atom()
b = ClassicalFormula(ClassicalLiteral(b_))
c__ = ASPBasicLiteral.make_literal('c')
c_ = c__.as_classical_atom()
c = ClassicalFormula(ClassicalLiteral(c_))
d__ = ASPBasicLiteral.make_literal('d')
d_ = d__.as_classical_atom()
d = ClassicalFormula(ClassicalLiteral(d_))
e__ = ASPBasicLiteral.make_literal('e')
e_ = e__.as_classical_atom()
e = ClassicalFormula(ClassicalLiteral(e_))


In [2827]:
def print_map_str(it: Iterable):
    print("{", ', '.join(map(str, it)), "}")

In [2828]:
K = {-(-c) / ClassicalBot(), -(a | c) / ClassicalBot(), -(b | c) / ClassicalBot()}
print_map_str(K)

{ a ∨ c, ¬c, b ∨ c }


In [2829]:
A = {-(-a) / ClassicalBot(), -(-b) / ClassicalBot(), -(d | e) / ClassicalBot(), -(-d | -e) / ClassicalBot()}
print_map_str(A)

{ d ∨ e, ¬b, ¬d ∨ ¬e, ¬a }


In [2830]:
def transform_to_lp_naive(K: KnowledgeBase, A: Set[Normally] = None,
                          alphabet: Optional[ClassicalAlphabet] = None) -> ASPProgram:
    K_ = materialized(K)
    if A is None:
        A = set()
    if alphabet is None:
        alphabet = {atom for statement in K for atom in statement.atoms} | {atom for statement in A for atom in
                                                                            statement.atoms} | {
                       ClassicalAtom("r_{}".format(str(clause))
                                     .replace(" ", "")
                                     .replace("∨", "_or_")
                                     .replace("→", "_implies_")
                                     .replace("¬", "_neg_")
                                     .replace("__", "_")) for clause in K_}
    i_rules = []
    for atom_ in sorted(alphabet):
        atom = abs(atom_)
        literal = ASPBasicLiteral(atom=ASPAtom(ASPFunction(atom.symbol)))
        rule_generate_positive = ASPNormalRule(literal, (-(~literal),))
        rule_generate_negative = ASPNormalRule(~literal, (-literal,))
        i_rules.append(rule_generate_positive)
        i_rules.append(rule_generate_negative)
    ii_rules = []
    A_ = materialized(A)
    for clause in sorted(A_, key=str):
        asp_bodies = (-clause).as_asp_bodies()
        for asp_body in asp_bodies:
            asp_body_ = tuple(literal if literal.is_pos else ~(-literal) for literal in asp_body)
            constraint = ASPIntegrityConstraint(body=asp_body_)
            ii_rules.append(constraint)

    iii_rules = []
    for clause in sorted(K_, key=str):
        asp_bodies = (-clause).as_asp_bodies()
        for asp_body in asp_bodies:
            asp_body_ = tuple(literal if literal.is_pos else ~(-literal) for literal in asp_body)
            head_ = ClassicalAtom("r_{}".format(str(clause))
                                  .replace(" ", "")
                                  .replace("∨", "_or_")
                                  .replace("→", "_implies_")
                                  .replace("¬", "_neg_")
                                  .replace("__", "_")
                                  )
            head = head_.as_asp_literal()
            rule = ASPNormalRule(head, asp_body_)
            iii_rules.append(rule)

    return ASPProgram((*i_rules, *ii_rules, *iii_rules))

In [2831]:
prg = transform_to_lp_naive(K, A)
print(prg.fmt('\n'))

a :- not -a.
-a :- not a.
b :- not -b.
-b :- not b.
c :- not -c.
-c :- not c.
d :- not -d.
-d :- not d.
e :- not -e.
-e :- not e.
r_a_or_c :- not -r_a_or_c.
-r_a_or_c :- not r_a_or_c.
r_b_or_c :- not -r_b_or_c.
-r_b_or_c :- not r_b_or_c.
r_neg_c :- not -r_neg_c.
-r_neg_c :- not r_neg_c.
:- -d, -e.
:- a.
:- b.
:- d, e.
r_a_or_c :- -a, -c.
r_b_or_c :- -b, -c.
r_neg_c :- c.


In [2832]:
tuple(asp_evaluate(prg, report=True));

Answer 1: { c e r_neg_c -a -b -d -r_a_or_c -r_b_or_c }
Answer 2: { c e r_b_or_c r_neg_c -a -b -d -r_a_or_c }
Answer 3: { c e r_a_or_c r_neg_c -a -b -d -r_b_or_c }
Answer 4: { c e r_a_or_c r_b_or_c r_neg_c -a -b -d }
Answer 5: { c d r_neg_c -a -b -e -r_a_or_c -r_b_or_c }
Answer 6: { c d r_a_or_c r_neg_c -a -b -e -r_b_or_c }
Answer 7: { c d r_b_or_c r_neg_c -a -b -e -r_a_or_c }
Answer 8: { c d r_a_or_c r_b_or_c r_neg_c -a -b -e }
Answer 9: { d r_a_or_c r_b_or_c -a -b -c -e -r_neg_c }
Answer 10: { d r_a_or_c r_b_or_c r_neg_c -a -b -c -e }
Answer 11: { e r_a_or_c r_b_or_c -a -b -c -d -r_neg_c }
Answer 12: { e r_a_or_c r_b_or_c r_neg_c -a -b -c -d }
SAT 12


In [2833]:
StatementRanking = Mapping[Number, Set[Normally]]


def statement_ranking(knowledge_base: KnowledgeBase) -> StatementRanking:
    E = {0: knowledge_base}
    rank = {}
    i = 0
    while True:
        E[i + 1] = set()
        for statement in E[i]:
            if entails(materialized(E[i]), -statement.left):
                E[i + 1].add(statement)
        if E[i] == E[i + 1]:
            break
        rank[i] = E[i] - E[i + 1]
        i = i + 1
    rank[float('inf')] = E[i]
    return rank




In [2834]:
def print_statement_ranking(statement_ranking: StatementRanking):
    r = len(statement_ranking) - (float('inf') in statement_ranking)
    pad = len(str(r)) + 1

    if len(statement_ranking) > 1:
        for i in range(r):
            print('{}:'.format(i).rjust(pad), end=' ')
            statements = statement_ranking[i]
            print_map_str(statements)

    if float('inf') in statement_ranking.keys():
        print('∞:'.rjust(pad), end=' ')
        statements = statement_ranking[float('inf')]
        print_map_str(statements)

In [2835]:
rk = statement_ranking(K)
print_statement_ranking(rk)

∞: { a ∨ c, ¬c, b ∨ c }


In [2836]:
f = ClassicalLiteral(ClassicalAtom('f'))  # flies
b = ClassicalLiteral(ClassicalAtom('b'))  # is a bird
p = ClassicalLiteral(ClassicalAtom('p'))  # is a pengiun
w = ClassicalLiteral(ClassicalAtom('w'))  # has wings
bot = ClassicalBot()  # Falsum
top = ClassicalTop()  # Verum
K = {-(p >> b) / bot, b / f, p / -f, b / w}
A = {p / f}
print_map_str(K)
print_map_str(materialized(K))

{ p → b, b |~ f, p |~ ¬f, b |~ w }
{ p → b, b → w, b → f, p → ¬f }


In [2837]:
rk = statement_ranking(K)
print_statement_ranking(rk)

0: { b |~ w, b |~ f }
1: { p |~ ¬f }
∞: { p → b }


In [2838]:
prg = transform_to_lp_naive(K)
print(prg.fmt('\n'))

b :- not -b.
-b :- not b.
f :- not -f.
-f :- not f.
p :- not -p.
-p :- not p.
r_b_implies_f :- not -r_b_implies_f.
-r_b_implies_f :- not r_b_implies_f.
r_b_implies_w :- not -r_b_implies_w.
-r_b_implies_w :- not r_b_implies_w.
r_p_implies_b :- not -r_p_implies_b.
-r_p_implies_b :- not r_p_implies_b.
r_p_implies_neg_f :- not -r_p_implies_neg_f.
-r_p_implies_neg_f :- not r_p_implies_neg_f.
w :- not -w.
-w :- not w.
r_b_implies_f :- b, -f.
r_b_implies_w :- b, -w.
r_p_implies_b :- p, -b.
r_p_implies_neg_f :- p, f.


In [2839]:
prg = transform_to_lp_naive(K, A)
print(prg.fmt('\n'))

b :- not -b.
-b :- not b.
f :- not -f.
-f :- not f.
p :- not -p.
-p :- not p.
r_b_implies_f :- not -r_b_implies_f.
-r_b_implies_f :- not r_b_implies_f.
r_b_implies_w :- not -r_b_implies_w.
-r_b_implies_w :- not r_b_implies_w.
r_p_implies_b :- not -r_p_implies_b.
-r_p_implies_b :- not r_p_implies_b.
r_p_implies_neg_f :- not -r_p_implies_neg_f.
-r_p_implies_neg_f :- not r_p_implies_neg_f.
w :- not -w.
-w :- not w.
:- p, -f.
r_b_implies_f :- b, -f.
r_b_implies_w :- b, -w.
r_p_implies_b :- p, -b.
r_p_implies_neg_f :- p, f.


In [2840]:
def transform_to_lp(K: KnowledgeBase, A: Set[Normally] = None,
                    alphabet: Optional[ClassicalAlphabet] = None) -> ASPProgram:
    K_ = materialized(K)
    if A is None:
        A = set()
    if alphabet is None:
        alphabet = {atom for statement in K for atom in statement.atoms} | {atom for statement in A for atom in
                                                                            statement.atoms} | {
                       ClassicalAtom("r_{}".format(str(clause))
                                     .replace(" ", "")
                                     .replace("∨", "_or_")
                                     .replace("→", "_implies_")
                                     .replace("¬", "_neg_")
                                     .replace("__", "_")) for clause in K_}
    rules = []
    for atom_ in sorted(alphabet):
        atom = abs(atom_)
        literal = ASPBasicLiteral(atom=ASPAtom(ASPFunction(atom.symbol)))
        rule_generate_positive = ASPNormalRule(literal, (-(~literal),))
        rule_generate_negative = ASPNormalRule(~literal, (-literal,))
        if rule_generate_positive not in rules:
            rules.append(rule_generate_positive)
        if rule_generate_negative not in rules:
            rules.append(rule_generate_negative)
    A_ = materialized(A)
    for clause in sorted(A_, key=str):
        asp_bodies = (-clause).as_asp_bodies()
        for asp_body in asp_bodies:
            asp_body_ = tuple(literal if literal.is_pos else ~(-literal) for literal in asp_body)
            constraint = ASPIntegrityConstraint(body=asp_body_)
            if constraint not in rules:
                rules.append(constraint)

    for clause in sorted(K_, key=str):
        asp_bodies = (-clause).as_asp_bodies()
        for asp_body in asp_bodies:
            asp_body_ = tuple(literal if literal.is_pos else ~(-literal) for literal in asp_body)
            head_ = ClassicalAtom("r_{}".format(str(clause))
                                  .replace(" ", "")
                                  .replace("∨", "_or_")
                                  .replace("→", "_implies_")
                                  .replace("¬", "_neg_")
                                  .replace("__", "_")
                                  )
            head = head_.as_asp_literal()
            rule = ASPNormalRule(head, asp_body_)
            if rule not in rules:
                rules.append(rule)
    for statement in sorted(A, key=str):
        asp_bodies = statement.left.as_asp_bodies()
        for asp_body in asp_bodies:
            asp_body_ = tuple(literal if literal.is_pos else ~(-literal) for literal in asp_body)
            constraint = ASPIntegrityConstraint(body=asp_body_)
            if constraint not in rules:
                rules.append(constraint)
    return ASPProgram(tuple(rules))

In [2841]:
K = {-(-c) / ClassicalBot(), -(a | c) / ClassicalBot(), -(b | c) / ClassicalBot()}
print_map_str(K)
A = {-(-a) / ClassicalBot(), -(-b) / ClassicalBot(), -(d | e) / ClassicalBot(), -(-d | -e) / ClassicalBot()}
print_map_str(A)

{ a ∨ c, ¬c, b ∨ c }
{ d ∨ e, ¬b, ¬d ∨ ¬e, ¬a }


In [2842]:
prg = transform_to_lp(K, A)
print(prg.fmt('\n'))

a :- not -a.
-a :- not a.
b :- not -b.
-b :- not b.
c :- not -c.
-c :- not c.
d :- not -d.
-d :- not d.
e :- not -e.
-e :- not e.
r_a_or_c :- not -r_a_or_c.
-r_a_or_c :- not r_a_or_c.
r_b_or_c :- not -r_b_or_c.
-r_b_or_c :- not r_b_or_c.
r_neg_c :- not -r_neg_c.
-r_neg_c :- not r_neg_c.
:- -d, -e.
:- a.
:- b.
:- d, e.
r_a_or_c :- -a, -c.
r_b_or_c :- -b, -c.
r_neg_c :- c.


In [2843]:
K = {-(p >> b) / bot, b / f, p / -f, b / w}
A = {p / f}
print_map_str(K)

{ p → b, b |~ f, p |~ ¬f, b |~ w }


In [2844]:
print_map_str(materialized(K))

{ p → b, b → w, b → f, p → ¬f }


In [2845]:
rk = statement_ranking(K)
print_statement_ranking(rk)

0: { b |~ w, b |~ f }
1: { p |~ ¬f }
∞: { p → b }


In [2846]:
prg = transform_to_lp(K)
print(prg.fmt('\n'))

b :- not -b.
-b :- not b.
f :- not -f.
-f :- not f.
p :- not -p.
-p :- not p.
r_b_implies_f :- not -r_b_implies_f.
-r_b_implies_f :- not r_b_implies_f.
r_b_implies_w :- not -r_b_implies_w.
-r_b_implies_w :- not r_b_implies_w.
r_p_implies_b :- not -r_p_implies_b.
-r_p_implies_b :- not r_p_implies_b.
r_p_implies_neg_f :- not -r_p_implies_neg_f.
-r_p_implies_neg_f :- not r_p_implies_neg_f.
w :- not -w.
-w :- not w.
r_b_implies_f :- b, -f.
r_b_implies_w :- b, -w.
r_p_implies_b :- p, -b.
r_p_implies_neg_f :- p, f.


In [2847]:
prg = transform_to_lp(K, A)
print(prg.fmt('\n'))

b :- not -b.
-b :- not b.
f :- not -f.
-f :- not f.
p :- not -p.
-p :- not p.
r_b_implies_f :- not -r_b_implies_f.
-r_b_implies_f :- not r_b_implies_f.
r_b_implies_w :- not -r_b_implies_w.
-r_b_implies_w :- not r_b_implies_w.
r_p_implies_b :- not -r_p_implies_b.
-r_p_implies_b :- not r_p_implies_b.
r_p_implies_neg_f :- not -r_p_implies_neg_f.
-r_p_implies_neg_f :- not r_p_implies_neg_f.
w :- not -w.
-w :- not w.
:- p, -f.
r_b_implies_f :- b, -f.
r_b_implies_w :- b, -w.
r_p_implies_b :- p, -b.
r_p_implies_neg_f :- p, f.
:- p.


In [2848]:
def clause_to_str(clause: ClassicalFormula):
    return str(clause).replace(" ", "").replace("∨", "_or_").replace("→", "_implies_").replace("¬", "_neg_").replace(
        "__", "_")


In [2849]:
def transform_to_metalp(K: KnowledgeBase, A: Set[Normally] = None,
                        alphabet: Optional[ClassicalAlphabet] = None) -> ASPProgram:
    K_ = materialized(K)
    if A is None:
        A = set()
    if alphabet is None:
        alphabet = {atom for statement in K for atom in statement.atoms} | {atom for statement in A for atom in
                                                                            statement.atoms}
    c = 0
    rules = []
    for atom_ in sorted(alphabet):
        atom = abs(atom_)
        fact_nr = ASPFunction('f_{}'.format(atom))
        fact_ = ASPFunction('rule', (fact_nr,))
        fact = ASPBasicLiteral(atom=ASPAtom(fact_))
        head_ = ASPFunction(atom.symbol)
        head = ASPBasicLiteral(atom=ASPAtom(ASPFunction('head', (fact_nr, head_))))
        meta_rule_rule = ASPNormalRule(fact)
        meta_rule_head = ASPNormalRule(head)
        if meta_rule_rule not in rules:
            rules.append(meta_rule_rule)
        if meta_rule_head not in rules:
            rules.append(meta_rule_head)

    for clause in sorted(K_, key=str):
        asp_bodies = (-clause).as_asp_bodies()
        rule_nr = ASPFunction('r_{}'.format(clause_to_str(clause)))
        rule_ = ASPFunction('rule', (rule_nr,))
        rule = ASPBasicLiteral(atom=ASPAtom(rule_))
        meta_rule_rule = ASPNormalRule(rule)
        if meta_rule_rule not in rules:
            rules.append(meta_rule_rule)
        blocked = ASPFunction('blocked', (rule_nr,))
        head = ASPFunction('head', (rule_nr, blocked,))
        meta_rule_head = ASPNormalRule(ASPBasicLiteral(atom=ASPAtom(head)))
        if meta_rule_head not in rules:
            rules.append(meta_rule_head)
            pass
        for asp_body in asp_bodies:
            for literal in asp_body:
                if literal.is_pos:
                    body_ = ASPFunction('bodyP', (rule_nr, literal.atom.symbol))
                else:
                    body_ = ASPFunction('bodyN', (rule_nr, literal.atom.symbol))
                meta_rule_body = ASPNormalRule(ASPBasicLiteral(atom=ASPAtom(body_)))
                if meta_rule_body not in rules:
                    rules.append(meta_rule_body)

    A_ = materialized(A)
    constraint_bodies = set()
    for clause in sorted(A_, key=str):
        asp_bodies = (-clause).as_asp_bodies()
        constraint_bodies.update(asp_bodies)
    for statement in sorted(A, key=str):
        if not statement.is_classical:
            asp_bodies = (-statement.left).as_asp_bodies()
            constraint_bodies.update(asp_bodies)

    for asp_body in constraint_bodies:
        c += 1
        constraint_nr = ASPFunction('c_{}'.format(c))
        constraint_ = ASPFunction('rule', (constraint_nr,))
        keep_ = ASPFunction('keep', (constraint_nr,))
        constraint = ASPBasicLiteral(atom=ASPAtom(constraint_))
        keep = ASPBasicLiteral(atom=ASPAtom(keep_))
        meta_rule_rule = ASPNormalRule(constraint)
        if meta_rule_rule not in rules:
            rules.append(meta_rule_rule)
        meta_rule_keep = ASPNormalRule(keep)
        if meta_rule_keep not in rules:
            rules.append(meta_rule_keep)
        for literal in asp_body:
            if literal.is_pos:
                body_ = ASPFunction('bodyP', (constraint_nr, literal.atom.symbol))
            else:
                body_ = ASPFunction('bodyN', (constraint_nr, literal.atom.symbol))
            meta_rule_body = ASPNormalRule(ASPBasicLiteral(atom=ASPAtom(body_)))
            if meta_rule_body not in rules:
                rules.append(meta_rule_body)

    return ASPProgram(tuple(rules))

In [2850]:
K = {-(-c) / ClassicalBot(), -(a | c) / ClassicalBot(), -(b | c) / ClassicalBot()}
print_map_str(K)

{ a ∨ c, ¬c, b ∨ c }


In [2851]:
A = {-(-a) / ClassicalBot(), -(-b) / ClassicalBot(), -(d | e) / ClassicalBot(), -(-d | -e) / ClassicalBot()}
print_map_str(A)

{ d ∨ e, ¬b, ¬d ∨ ¬e, ¬a }


In [2852]:
prg = transform_to_metalp(K, A)
print(prg.fmt('\n'))

rule(f_a).
head(f_a,a).
rule(f_b).
head(f_b,b).
rule(f_c).
head(f_c,c).
rule(f_d).
head(f_d,d).
rule(f_e).
head(f_e,e).
rule(r_a_or_c).
head(r_a_or_c,blocked(r_a_or_c)).
bodyN(r_a_or_c,a).
bodyN(r_a_or_c,c).
rule(r_b_or_c).
head(r_b_or_c,blocked(r_b_or_c)).
bodyN(r_b_or_c,b).
bodyN(r_b_or_c,c).
rule(r__neg_c).
head(r__neg_c,blocked(r__neg_c)).
bodyP(r__neg_c,c).
rule(c_1).
keep(c_1).
bodyP(c_1,d).
bodyP(c_1,e).
rule(c_2).
keep(c_2).
bodyP(c_2,a).
rule(c_3).
keep(c_3).
bodyP(c_3,b).
rule(c_4).
keep(c_4).
bodyN(c_4,d).
bodyN(c_4,e).


In [2853]:
debug_meta_str = """

{ remove(R) } :- rule(R).

:- remove(R), keep(R).

:- not remove(R), head(R,blocked(R)), hold(blocked(R)).

hold(H) : head(R,H) :-
  rule(R),
  not remove(R),
      hold(L) : bodyP(R,L);
  not hold(L) : bodyN(R,L).

#show.
#show H: hold(H).
#show remove/1.

"""

In [2854]:
tuple(asp_evaluate(prg, debug_meta_str, report=True));

Answer 1: { c e remove(f_a) remove(f_b) remove(f_d) remove(r__neg_c) }
Answer 2: { c e remove(f_a) remove(f_b) remove(f_d) remove(r__neg_c) remove(r_b_or_c) }
Answer 3: { c d remove(f_a) remove(f_b) remove(f_e) remove(r__neg_c) }
Answer 4: { c d remove(f_a) remove(f_b) remove(f_e) remove(r__neg_c) remove(r_b_or_c) }
Answer 5: { c e remove(f_a) remove(f_b) remove(f_d) remove(r__neg_c) remove(r_a_or_c) }
Answer 6: { c d remove(f_a) remove(f_b) remove(f_e) remove(r__neg_c) remove(r_a_or_c) }
Answer 7: { c e remove(f_a) remove(f_b) remove(f_d) remove(r__neg_c) remove(r_a_or_c) remove(r_b_or_c) }
Answer 8: { c d remove(f_a) remove(f_b) remove(f_e) remove(r__neg_c) remove(r_a_or_c) remove(r_b_or_c) }
Answer 9: { e remove(f_a) remove(f_b) remove(f_c) remove(f_d) remove(r__neg_c) remove(r_a_or_c) remove(r_b_or_c) }
Answer 10: { d remove(f_a) remove(f_b) remove(f_c) remove(f_e) remove(r__neg_c) remove(r_a_or_c) remove(r_b_or_c) }
Answer 11: { e remove(f_a) remove(f_b) remove(f_c) remove(f_d) re

In [2855]:
K = {-(p >> b) / bot, b / f, p / -f, b / w}
A = {p / f}
print_map_str(K)

{ p → b, b |~ f, p |~ ¬f, b |~ w }


In [2856]:
print_map_str(materialized(K))

{ p → b, b → w, b → f, p → ¬f }


In [2857]:
rk = statement_ranking(K)
print_statement_ranking(rk)

0: { b |~ w, b |~ f }
1: { p |~ ¬f }
∞: { p → b }


In [2858]:
prg = transform_to_metalp(K, A)
print(prg.fmt('\n'))

rule(f_b).
head(f_b,b).
rule(f_f).
head(f_f,f).
rule(f_p).
head(f_p,p).
rule(f_w).
head(f_w,w).
rule(r_b_implies_f).
head(r_b_implies_f,blocked(r_b_implies_f)).
bodyP(r_b_implies_f,b).
bodyN(r_b_implies_f,f).
rule(r_b_implies_w).
head(r_b_implies_w,blocked(r_b_implies_w)).
bodyP(r_b_implies_w,b).
bodyN(r_b_implies_w,w).
rule(r_p_implies_b).
head(r_p_implies_b,blocked(r_p_implies_b)).
bodyP(r_p_implies_b,p).
bodyN(r_p_implies_b,b).
rule(r_p_implies_neg_f).
head(r_p_implies_neg_f,blocked(r_p_implies_neg_f)).
bodyP(r_p_implies_neg_f,p).
bodyP(r_p_implies_neg_f,f).
rule(c_1).
keep(c_1).
bodyP(c_1,p).
bodyN(c_1,f).
rule(c_2).
keep(c_2).
bodyN(c_2,p).


In [2859]:
tuple(asp_evaluate(prg, debug_meta_str, report=True));

Answer 1: { b f p w remove(r_p_implies_neg_f) }
Answer 2: { b f p w remove(r_p_implies_b) remove(r_p_implies_neg_f) }
Answer 3: { b f p w remove(r_b_implies_f) remove(r_p_implies_neg_f) }
Answer 4: { b f p w remove(r_b_implies_f) remove(r_p_implies_b) remove(r_p_implies_neg_f) }
Answer 5: { b f p w remove(r_b_implies_w) remove(r_p_implies_neg_f) }
Answer 6: { b f p remove(f_w) remove(r_b_implies_w) remove(r_p_implies_neg_f) }
Answer 7: { b f p w remove(r_b_implies_f) remove(r_b_implies_w) remove(r_p_implies_neg_f) }
Answer 8: { b f p remove(f_w) remove(r_b_implies_f) remove(r_b_implies_w) remove(r_p_implies_neg_f) }
Answer 9: { b f p w remove(r_b_implies_w) remove(r_p_implies_b) remove(r_p_implies_neg_f) }
Answer 10: { b f p remove(f_w) remove(r_b_implies_w) remove(r_p_implies_b) remove(r_p_implies_neg_f) }
Answer 11: { b f p w remove(r_b_implies_f) remove(r_b_implies_w) remove(r_p_implies_b) remove(r_p_implies_neg_f) }
Answer 12: { b f p remove(f_w) remove(r_b_implies_f) remove(r_b_im

In [ ]:
def generate_meta_rules(K: KnowledgeBase):
    r = 0
    meta_rules = []
    for clause in K:
        r += 1
        rule_nr = ASPFunction('r_{}'.format(r))
        rule = ASPNormalRule(ASPBasicLiteral(atom=ASPAtom(ASPFunction('rule', (rule_nr,)))))
        meta_rules.append(rule)
        head_formula = clause.right
        body_formula = clause.left
        asp_heads = head_formula.as_asp_heads()
        asp_bodies = body_formula.as_asp_bodies()
        for asp_head in asp_heads:
            # TODO each head is a new rule
            for asp_body in asp_bodies:
                for asp_head_literal in asp_head:
                    assert isinstance(asp_head_literal, ASPBasicLiteral)
                    asp_head_symbol = asp_head_literal.atom.symbol
                    meta_head = ASPNormalRule(ASPBasicLiteral(atom=ASPAtom(ASPFunction('head', (rule_nr, asp_head_symbol)))))
    # TODO: Generate body

In [2860]:
def generate_prioritization_meta_rules(K: KnowledgeBase, rk: Optional[StatementRanking] = None):
    if rk is None:
        rk = statement_ranking(K)
    meta_rules = []
    if float('inf') in rk:
        for higher in rk[float('inf')]:
            higher_ = ASPFunction(clause_to_str(higher.materialize()))
            for lower in rk[len(rk)-2]:
                lower_ = ASPFunction(clause_to_str(lower.materialize()))
                pr_ = ASPFunction('pr', (higher_, lower_))
                pr = ASPBasicLiteral(atom=ASPAtom(pr_))
                meta_rule = ASPNormalRule(pr)
                meta_rules.append(meta_rule)



    for i in range(len(rk)):
        if i in rk:
            for higher in rk[i]:
                higher_ = ASPFunction(clause_to_str(higher.materialize()))
                if i-1 in rk:
                    for lower in rk[i-1]:
                        lower_ = ASPFunction(clause_to_str(lower.materialize()))
                        pr_ = ASPFunction('pr', (higher_, lower_))
                        pr = ASPBasicLiteral(atom=ASPAtom(pr_))
                        meta_rule = ASPNormalRule(pr)
                        meta_rules.append(meta_rule)
    return meta_rules



In [2861]:
prioritization_meta_rules = generate_prioritization_meta_rules(K)
print_map_str(prioritization_meta_rules)

{ pr(p_implies_b,p_implies_neg_f)., pr(p_implies_neg_f,b_implies_w)., pr(p_implies_neg_f,b_implies_f). }


In [2862]:
meta_str = """

hold(H) : head(R,H) :-
  rule(R),
  not remove(R),
      hold(L) : bodyP(R,L);
  not hold(L) : bodyN(R,L).

#show.
#show H: hold(H).
#show remove/1.

"""

In [2863]:
b_preferred_str = """

% For full prioritization: refine pr to a total ordering.
pr(X,Y) ; pr(Y,X) :- rule(X), rule(Y), X != Y.
pr(X,Z) :- pr(X,Y), pr(Y,Z).
:- pr(X,X).

% Check dual reduct: Build sets S_i, use rule ids as indices i.
% lit(X,r) means that the literal x occurs in the set S_r.
lit(L,R) :-
  head(R,L),
  hold(L) : bodyP(R,L);
  not defeat_local(R),
  not hold(L).
lit(L,R) :-
  head(R,L),
  hold(L) : bodyP(R,L);
  not defeat_local(R),
  not defeat_global(R).

defeat_local(R) :- bodyN(R,L), lit(L,R'), pr(R',R).
defeat_global(R) :- bodyN(R,L), hold(L).

% Include literal into CP(.).
in_CP(L) :- lit(L,R).
:- in_CP(L), not hold(L).
:- hold(L), not in_CP(L).

"""

In [2864]:
tuple(asp_evaluate(
    prg,
    '\n'.join(map(str, prioritization_meta_rules)),
    b_preferred_str,
    debug_meta_str,
    report=True));

UNSAT
